In [68]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

import os
import pandas as pd

In [69]:
def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB') 
        
# resize and take the center part of image to what our model expects
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    # unsqeeze converts single image to batch of 1
    return transf(img).unsqueeze(0)
        


In [70]:
pic_list=[]
for root, dirs, files in os.walk("memes/defaults/images/", topdown=True, onerror=None, followlinks=False):
    for filename in files:
        pic_list.append(filename)
        
        
pic_list=pic_list

In [71]:
model = models.inception_v3(pretrained=True)

/home/luca/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/luca/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [72]:
idx2label, cls2label, cls2idx = [], {}, {}
with open(os.path.abspath("memes/defaults/extras/imagenet_class_index.json"), 'r') as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
    cls2label = {class_idx[str(k)][0]: class_idx[str(k)][1] for k in range(len(class_idx))}
    cls2idx = {class_idx[str(k)][0]: k for k in range(len(class_idx))}    

In [73]:
folder_path="memes/defaults/images/"

df=[]
for i, img_filename in enumerate(pic_list):
    img_path = os.path.join(folder_path, img_filename)
    print(f'Image {i} out of {len(pic_list)}\n{img_path}')
    img = get_image(img_path)
    
    img_t = get_input_tensors(img)  
    model.eval()    
    logits = model(img_t)   
        
    probs = F.softmax(logits, dim=1)    
    probs5 = probs.topk(5)  
    obj=tuple((p,c, idx2label[c]) for p, c in zip(probs5[0][0].detach().numpy(), probs5[1][0].detach().numpy()))
    
    for k in obj:
        
        obj_dict = {
            "img": img_filename,
            "prob":k[0],
            "label_id":k[1],
            "label": k[2]
        }
        df.append(obj_dict)
    

df = pd.DataFrame(df)
df
    
df.to_csv("Images_Objects.csv")

Image 0 out of 3544
memes/defaults/images/covid_memes_5594.png
Image 1 out of 3544
memes/defaults/images/covid_memes_2353.png
Image 2 out of 3544
memes/defaults/images/covid_memes_3945.png
Image 3 out of 3544
memes/defaults/images/covid_memes_722.png
Image 4 out of 3544
memes/defaults/images/covid_memes_807.png
Image 5 out of 3544
memes/defaults/images/covid_memes_1602.png
Image 6 out of 3544
memes/defaults/images/covid_memes_1053.png
Image 7 out of 3544
memes/defaults/images/covid_memes_1769.png
Image 8 out of 3544
memes/defaults/images/covid_memes_5425.png
Image 9 out of 3544
memes/defaults/images/covid_memes_1052.png
Image 10 out of 3544
memes/defaults/images/covid_memes_801.png
Image 11 out of 3544
memes/defaults/images/covid_memes_1587.png
Image 12 out of 3544
memes/defaults/images/covid_memes_4315.png
Image 13 out of 3544
memes/defaults/images/covid_memes_3411.png
Image 14 out of 3544
memes/defaults/images/covid_memes_2602.png
Image 15 out of 3544
memes/defaults/images/covid_meme